# CrewAI with Remote MCP Connection Sample

This notebook demonstrates how to use CrewAI to connect to a remote MCP (Model Context Protocol) server and utilize its tools.

In [ ]:
!pip install crewai 'crewai-tools[mcp]' python-dotenv

Import necessary libraries and set up environment variables.

In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import MCPServerAdapter
from dotenv import load_dotenv

In [ ]:
load_dotenv()
# Placeholder for OPENAI_API_KEY or other LLM provider keys
# Ensure you have a .env file in the root of this repository with your API key
# For example:
# OPENAI_API_KEY="sk-..." 
# OPENAI_MODEL_NAME="gpt-4o-mini"

## 1. Configure MCP Server Connection

You need to define the connection parameters for your remote MCP server.
This typically involves specifying the URL and the transport mechanism (`streamable-http` or `sse`).

**Example using a local Streamable HTTP server (like `hello_http_server.py` from the `crewai-mcp-demo` repository):**
If you were running the `hello_http_server.py` from the demo, it would be:
`server_params = {"url": "http://localhost:8001/mcp", "transport": "streamable-http"}`

**For your actual remote MCP server, replace the values below with your server's details.**

In [ ]:
# TODO: Configure these parameters for your remote MCP server
# Option 1: Streamable HTTP
server_params = {
    "url": "YOUR_REMOTE_MCP_SERVER_URL/mcp",  # Replace with your server's URL (often ends with /mcp)
    "transport": "streamable-http"
}

# Option 2: SSE
# server_params = {
#     "url": "YOUR_REMOTE_MCP_SSE_ENDPOINT_URL",  # Replace with your server's SSE endpoint URL
#     "transport": "sse" # SSE transport typically doesn't need 'transport' explicitly if URL is specific
# }

# Example for testing with a local dummy server (if you have one running, e.g., hello_http_server.py)
# server_params = {"url": "http://localhost:8001/mcp", "transport": "streamable-http"}

print(f"MCP Server Params: {server_params}")

## 2. Define Agent and Task to Use MCP Tools

Now we'll create a CrewAI agent and provide it with the tools made available by the MCP server.
The task will instruct the agent to use one of these tools.

**Note:** The agent's role, goal, backstory, and the task's description will be generic. 
You should customize these based on the actual capabilities of your MCP server and the tools it provides.

In [ ]:
# Placeholder for agent and task definitions
# We'll attempt to connect to the MCP server and list its tools first.
# If successful, we'll define an agent that uses these tools.

mcp_agent = None
mcp_task = None

try:
    with MCPServerAdapter(server_params) as mcp_tools:
        available_tools = [tool.name for tool in mcp_tools]
        print(f"Available tools from MCP server: {available_tools}")

        if not mcp_tools:
            print("No tools found or connection failed. Please check your MCP server and `server_params`.")
            print("Skipping agent and task creation.")
        else:
            # Define an agent that uses the MCP tools
            mcp_agent = Agent(
                role="Remote Service Interactor",
                goal=f"Utilize tools from the MCP server at {server_params.get('url')} to accomplish tasks.",
                backstory="I am an AI agent designed to connect to remote MCP servers and leverage their functionalities through CrewAI.",
                tools=mcp_tools,  # Assign the fetched MCP tools to the agent
                allow_delegation=False,
                verbose=True,
                # Optional: configure LLM if not using default OpenAI
                # llm=OpenAI(api_key=os.getenv("OPENAI_API_KEY"), model_name=os.getenv("OPENAI_MODEL_NAME")) 
            )

            # Define a task for the agent
            # TODO: Customize the task description and expected_output based on your MCP server's tools and desired action.
            # You might want to prompt the user for the tool name and specific inputs.
            # For this example, we'll make a generic task. If specific tools are known, it's better to be explicit.
            
            task_description = "Access the remote MCP server and list the available high-level capabilities or perform a default action."
            expected_output = "A summary of capabilities or the result of the default action from the MCP server."

            if available_tools:
                # If specific tools are discovered, you could make the task more targeted.
                # For example, if a 'get_status' tool exists:
                # task_description = f"Use the '{available_tools[0]}' tool to get the current status from the MCP server."
                # expected_output = "The current status information from the MCP server."
                # For now, let's assume the agent will figure out how to interact based on the generic tools.
                task_description = f"Interact with the MCP server using one of its available tools ({', '.join(available_tools)}) to get a general status or introductory message."
                expected_output = "A response from the MCP server based on the interaction."


            mcp_task = Task(
                description=task_description,
                expected_output=expected_output,
                agent=mcp_agent
            )
            
            print("Agent and Task defined successfully.")
            print(f"Agent: {mcp_agent.role}")
            print(f"Task: {mcp_task.description}")

except Exception as e:
    print(f"An error occurred while trying to connect to the MCP server or define the agent/task: {e}")
    print("Please ensure your MCP server is running and `server_params` are correctly configured.")
    print("If you are using a local server for testing (e.g., from crewai-mcp-demo), make sure it's started.")

## 3. Handling Connection, Authentication, and Errors

The `MCPServerAdapter` handles the basic connection to the MCP server using the `server_params` you provide. The `try...except` block in the previous step is set up to catch common issues like:
*   The MCP server being unreachable.
*   Incorrect `server_params` (e.g., wrong URL or transport type).
*   Issues during the MCP handshake.

**Authentication:**

*   If your remote MCP server requires authentication (e.g., API keys, tokens), the method for providing these credentials depends on the MCP server's implementation and the tools it exposes.
*   **Tool-Based Authentication:** Some tools might accept credentials as part of their input schema when they are called by the agent. You would need to understand the specific requirements of your MCP server's tools.
*   **Custom Headers/Parameters:** If the MCP server expects authentication information via HTTP headers (like an `Authorization` header) or specific query parameters, and if the `MCPServerAdapter` or the underlying MCP client library supports passing these through `server_params`, you would include them there. The current `crewai-tools` documentation for `MCPServerAdapter` doesn't explicitly detail adding custom headers for HTTP/SSE transports directly in `server_params`. This might require the MCP server itself to be designed to accept auth through standard mechanisms its tools can access, or potentially a more advanced setup if direct header injection via the adapter is needed but not supported.
*   **Environment Variables:** For sensitive data like API keys, always prefer using environment variables (as shown for the LLM API key) and have your MCP tool or server retrieve them securely.

Ensure your MCP server is configured to handle authentication appropriately and that your CrewAI agent/task provides any necessary credentials in a secure manner, likely when invoking a specific tool that requires them.

The notebook will proceed to define a Crew and attempt to run it. If the connection failed or no tools were found in the previous step, the `mcp_agent` and `mcp_task` will be `None`, and the Crew execution will be skipped.

## 4. Run the Crew and Demonstrate Interaction

If the MCP server connection was successful and the agent and task were defined, we can now create a Crew and run it.
The crew will execute the task, which involves the agent using the tools from the remote MCP server.

**Note:** The actual interaction and output will depend heavily on:
1.  Your MCP server's tools and how they respond.
2.  The LLM's ability to understand how to use the provided tools based on their descriptions.
3.  The specifics of the task you defined for the agent.

In [ ]:
# Only proceed if the agent and task were successfully created
if mcp_agent and mcp_task:
    # Create the crew
    mcp_crew = Crew(
        agents=[mcp_agent],
        tasks=[mcp_task],
        process=Process.sequential,  # Sequential process for this example
        verbose=2  # Provides detailed output of the crew's execution
    )

    # Kick off the crew's execution
    # TODO: If your task requires specific inputs, provide them in the `inputs` dictionary.
    # For the generic task defined, we might not need specific inputs, 
    # but if your customized task uses placeholders like {topic}, provide them here.
    # Example: inputs={"topic": "general status"}
    print("\nStarting Crew execution...")
    try:
        result = mcp_crew.kickoff(inputs={}) # Add inputs if your task requires them
        
        print("\n--------------------------------------------------")
        print("Crew Execution Finished.")
        print("Result from MCP interaction via CrewAI:")
        print(result)
        print("--------------------------------------------------")

    except Exception as e:
        print(f"An error occurred during crew execution: {e}")
        print("This could be due to issues with the LLM, the tools themselves, or the way the task is defined.")

else:
    print("Skipping Crew execution because the agent and/or task were not properly initialized.")
    print("Please check previous steps for errors, especially MCP server connection and tool discovery.")

## 5. Summary and Next Steps

This notebook has demonstrated the foundational steps to connect CrewAI to a remote MCP server:

1.  **Setup:** Installing necessary libraries (`crewai`, `crewai-tools[mcp]`).
2.  **Configuration:**
    *   Setting up environment variables for LLM API keys (e.g., `OPENAI_API_KEY`).
    *   Defining `server_params` with your remote MCP server's URL and transport mechanism. This is a **critical step** for you to customize.
3.  **MCP Connection & Tool Integration:**
    *   Using `MCPServerAdapter` to connect to your MCP server and automatically make its tools available to CrewAI.
4.  **Agent and Task Definition:**
    *   Creating a CrewAI `Agent` equipped with the tools from your MCP server.
    *   Defining a `Task` that instructs the agent on what to achieve using these tools.
    *   Both the agent (role, goal, backstory) and task (description, expected_output) are generic in this sample and **must be customized** to match your MCP server's capabilities and your specific objectives. The more descriptive your tool names and descriptions are on the MCP server, and the clearer your task is, the better the LLM will be at using them.
5.  **Crew Execution:**
    *   Forming a `Crew` and `kickoff()` its execution to see the agent interact with the MCP server.
    *   The results depend on the successful integration and the nature of the tools on your MCP server.

**Key Customization Points for You:**

*   **`server_params`**: Update this with your actual remote MCP server details.
*   **Agent Definition**: Tailor the `role`, `goal`, and `backstory` of the `mcp_agent`.
*   **Task Definition**: Modify the `description` and `expected_output` of the `mcp_task` to be specific to what you want to achieve with your MCP server's tools. If your task requires inputs, provide them in the `crew.kickoff(inputs={...})` call.
*   **LLM Configuration**: If you're not using OpenAI or want to specify a different model, configure the `llm` parameter when creating your `Agent`.
*   **Authentication**: Ensure any authentication required by your MCP server is handled (as discussed in section 3).

By modifying these sections, you can adapt this notebook to work with your specific remote MCP environment. Remember to consult the documentation for your MCP server to understand the tools it provides and how to interact with them.